# Import Libraries

In [1]:
import pandas as pd
from itertools import zip_longest
from bs4 import BeautifulSoup
from selenium import webdriver

from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time

C:\Users\Gilbert KH\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# code to force show all column
pd.set_option('display.max_columns', None)

# max column width
pd.set_option('display.max_colwidth', 100)

# Load Dataset

In [3]:
df = pd.read_csv('./datasets/rumah123_hyperlink.csv')

# Scrap Web Per Pages

In [4]:
#init webdriver & test BeautifulSoup
driver = webdriver.Chrome()

url="https://www.rumah123.com/jual/rumah/?q=Balikpapan"
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser") #Menentukan parser HTML yang ingin kita gunakan.
print(soup.prettify()[:700]) # memberikan representasi visual dari pohon parse yang dibuat dari konten HTML mentah.

In [ ]:
# prepare variable to save the datas
id = []
judul = []
harga = []
lokasi = []
kamar_tidur = []
kamar_mandi = []
luas_tanah = []
luas_bangunan = []
carport = []
tipe_properti = []
sertifikat = []
daya_listrik = []

kamar_pembantu = []
dapur = []
ruang_makan = []
ruang_tamu = []
kondisi_perabotan = []

# Exterior
jumlah_lantai = []
pemandangan = []

# tentang properti
terjangkau_internet = []
lebar_jalan = []
sumber_air = []
hook = []
kondisi_properti = []
tipe_iklan = []
id_iklan = []

### Changed Logic
1. First initial labels of all possible data
2. Create a dictionary based on the labels
3. Create a Loop to scrap the data and store it in the dictionary
    - First, fill the dictionary with NaN for all key so that when we didn't found the data, it still in the same shape
    - Loop 1: To get the labels1 data (Kamar Tidur, Kamar Mandi, etc)
    - Loop 2: To get the labels2 data (Luas Bangunan, Luas Tanah, etc)

In [ ]:
# clean dictionary
data1 = {}
data2 = {}
harga = []
judul = []
lokasi = []
harga = [np.nan]*1467
judul = [np.nan]*1467
lokasi = [np.nan]*1467

# define the labels
labels1 = ['Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan', 'Carport', 'Tipe Properti', 'Sertifikat', 'Daya Listrik']
labels2 = ['Kamar Pembantu', 'Dapur', 'Ruang Makan', 'Ruang Tamu', 'Kondisi Perabotan', 'Jumlah Lantai', 'Pemandangan', 'Terjangkau Internet', 'Lebar Jalan', 'Sumber Air', 'Hook', 'Kondisi Properti', 'Tipe Iklan']

# create dictionaries to store the labels and their values
data1 = {label: [np.nan]*1467 for label in labels1}
data2 = {label: [np.nan]*1467 for label in labels2}

loop = 501
print("Progresss: ")

# traverse the hyperlink to get the data needed (test 10 data first)
for i in df['hyperlink'][501:1000]:
    url=i
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # generate random number between 1 and 3
    rand = np.random.randint(1, 17)

    # short pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 5) == 0) and (loop != 0):
        pause = 25 - rand
        time.sleep(pause)
        # print(f'Long Pause for {pause} seconds...')
    
    # Long pause if loop is a multiple of 19, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 19) == 0) and (loop != 0):
        pause = 79 - rand
        print(f'Long Pause for {pause} seconds...')
        time.sleep(pause)
    
    # Super long pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 98) == 0) and (loop != 0):
        pause = 820 - rand
        print(f'Super Long Pause for {pause} seconds...')
        time.sleep(pause)
    
    
    get_price_iklan = soup.find_all('div', class_="r123-listing-summary__price")
    # get the span inside the div
    for i in get_price_iklan:
        get_price_iklan = i.find('span')  # use find instead of find_all
        harga[loop] = get_price_iklan.text

    judul_elements = soup.find_all('h1', {"class":"r123-listing-summary__header-container-title"})
    if judul_elements:
        judul[loop] = judul_elements[0].get_text()

    lokasi_elements = soup.find_all('div', {"class":"r123-listing-summary__header-container-address"})
    if lokasi_elements:
        lokasi[loop] = lokasi_elements[0].get_text()
    
    # ----------------- #

    # first loop
    spec_items = soup.find_all('div', class_="listing-specification-v2__item")
    for item in spec_items:
        label = item.find('span', class_="listing-specification-v2__item-label").text
        value = item.find('span', class_="listing-specification-v2__item-value").text

        # if the label is in the first dictionary, insert the value at the index of the current loop
        if label in data1:
            data1[label][loop] = value

    # second loop
    interior_items = soup.find_all('div', class_="ui-molecule-accordion-r123")
    for item in interior_items:
        spec_items = item.find_all('div', class_="listing-specification-v2__item")
        for spec_item in spec_items:
            label = spec_item.find('span', class_="listing-specification-v2__item-label").text
            value = spec_item.find('span', class_="listing-specification-v2__item-value").text

            # if the label is in the second dictionary, insert the value at the index of the current loop
            if label in data2:
                data2[label][loop] = value
   
    #----------------- #
    loop += 1
    
    # print loop progress total number of link divided by hyperlinks
    print(f'Done: {loop} / {df["hyperlink"].size}')    

Progresss: 
Done: 502 / 1467
Done: 503 / 1467
Done: 504 / 1467
Done: 505 / 1467
Done: 506 / 1467
Done: 507 / 1467
Done: 508 / 1467
Done: 509 / 1467
Done: 510 / 1467
Done: 511 / 1467
Done: 512 / 1467
Done: 513 / 1467
Long Pause for 73 seconds...
Done: 514 / 1467
Done: 515 / 1467
Done: 516 / 1467
Done: 517 / 1467
Done: 518 / 1467
Done: 519 / 1467
Done: 520 / 1467
Done: 521 / 1467
Done: 522 / 1467
Done: 523 / 1467
Done: 524 / 1467
Done: 525 / 1467
Done: 526 / 1467
Done: 527 / 1467
Done: 528 / 1467
Done: 529 / 1467
Done: 530 / 1467
Done: 531 / 1467
Done: 532 / 1467
Long Pause for 71 seconds...
Done: 533 / 1467
Done: 534 / 1467
Done: 535 / 1467
Done: 536 / 1467
Done: 537 / 1467
Done: 538 / 1467
Done: 539 / 1467
Done: 540 / 1467
Done: 541 / 1467
Done: 542 / 1467
Done: 543 / 1467
Done: 544 / 1467
Done: 545 / 1467
Done: 546 / 1467
Done: 547 / 1467
Done: 548 / 1467
Done: 549 / 1467
Done: 550 / 1467
Done: 551 / 1467
Long Pause for 69 seconds...
Done: 552 / 1467
Done: 553 / 1467
Done: 554 / 1467


In [ ]:
# get the intersection of the keys in data1 and data2
# duplicate_labels = set(data1.keys()) & set(data2.keys())

# # for each duplicate label, remove it from data2
# for label in duplicate_labels:
#     del data2[label]

# convert the dictionaries to dataframes
judull = pd.DataFrame(judul, columns=['Judul'])
hargaa = pd.DataFrame(harga, columns=['Harga'])
lokasia = pd.DataFrame(lokasi, columns=['Lokasi'])

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# concatenate the dataframes
df_fix = pd.concat([judull, hargaa, lokasia, df1, df2], axis=1)
print(df_fix.shape)

# drop all rows Judul NaN
df_fix = df_fix.dropna(subset=['Judul'])

df_fix

(1467, 24)


,Judul,Harga,Lokasi,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
685,"DI RUMAH 2 1/2 LANTAI STRATEGIS PINGGIR JALAN RAYA DR SUTOMO (KARANG REJO), BALIKPAPAN","Rp 2,2 Miliar","Karang Rejo, Balikpapan",7,7,176 m²,240 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Tidak,Tidak,Unfurnished,3,Perkotaan,Tidak,2 Mobil,NaN,Tidak,Bagus,Dijual
688,BALIKPAPAN RUMAH DI PERUMAHAN KUTAI HILLS (HM173)TURUN HARGA 750JT,Rp 850 Juta,"Balikpapan Utara, Balikpapan",3,2,120 m²,90 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,NaN,Perkotaan,Tidak,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
689,BALIKPAPAN RUMAH PERUMAHAN BUKIT DAMAI INDAH BDI (HM174),"Rp 1,9 Miliar","Balikpapan Selatan, Balikpapan",4,3,214 m²,120 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,Tidak,Tidak,Semi Furnished,NaN,Perkotaan,Tidak,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
690,73 Balikpapan Bima Jamrud Rumah Dalam Perumahan Padat Penduduk,Rp 925 Juta,"Balikpapan Selatan, Balikpapan",2,1,92 m²,60 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
691,"RUMAH BANGUNAN BARU,DI PERUMAHAN GRAND CITY",Rp 950 Juta,"Balikpapan Tengah, Balikpapan",2,2,80 m²,100 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
692,Rumah elit gaya minimalis modern sejuk diperumahan megah,"Rp 1,97 Miliar","Balikpapan Baru, Balikpapan",3,2,162 m²,110 m²,1,Rumah,SHM - Sertifikat Hak Milik,5500 Watt,NaN,NaN,Ya,Ya,Semi Furnished,1,Pemukiman Warga,Ya,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
693,"DIJUAL RUMAH BALIKPAPAN BARU ARIZONA, BALIKPAPAN","Rp 1,2 Miliar","Balikpapan Baru, Balikpapan",2,2,140 m²,90 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Tidak,Tidak,Unfurnished,1,Pemukiman Warga,Tidak,2 Mobil,NaN,Tidak,Bagus,Dijual
694,"DI RUMAH BARU READY STOCK DAN SIAP HUNI, LOKASI SANGAT STRATEGIS",Rp 405 Juta,"Balikpapan Utara, Balikpapan",2,1,96 m²,45 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
695,RUMAH ETNIK DI BATU AMPAR DENGAN KONSEP CONTTAGE ASRI DAN NYAMAN,"Rp 4,25 Miliar","Batu Ampar, Balikpapan",5,6,770 m²,350 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
696,DIJUAL RUMAH KONTRAKAN 2 LANTAI 4 PINTU DEKAT PINTU TOL & DEKAT DAERAH INDUSTRI BATAKAN BALIKPAPAN,Rp 1 Miliar,"Batakan, Balikpapan",8,4,135 m²,120 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,900 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual


In [ ]:
# to csv
df_fix.to_csv('./datasets/rumah123_data_single_raw_fix_part_2.csv', index=True)

In [ ]:
# full inner join on judul and Judul delete the rest that not match
df_fix_combine = pd.merge(df, df_fix, left_on='judul', right_on='Judul', how='inner')

# drop where Judul is NaN
df_fix_combine = df_fix_combine.dropna(subset=['Judul'])

In [ ]:
df_fix_combine.sample(10)

,judul,hyperlink,img_hyperlinks,Judul,Harga,Lokasi,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
0,"RUMAH BANGUNAN BARU,DI PERUMAHAN GRAND CITY",https://www.rumah123.com//properti/balikpapan/hos14169683/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14169683/original/hos14169683-rumah-di...,"RUMAH BANGUNAN BARU,DI PERUMAHAN GRAND CITY",Rp 950 Juta,"Balikpapan Tengah, Balikpapan",2,2,80 m²,100 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1,"RUMAH BANGUNAN BARU,DI PERUMAHAN GRAND CITY",https://www.rumah123.com//properti/balikpapan/hos14152008/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14152008/original/hos14152008-rumah-di...,"RUMAH BANGUNAN BARU,DI PERUMAHAN GRAND CITY",Rp 950 Juta,"Balikpapan Tengah, Balikpapan",2,2,80 m²,100 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
2,"DI RUMAH 2 1/2 LANTAI STRATEGIS PINGGIR JALAN RAYA DR SUTOMO (KARANG REJO), BALIKPAPAN",https://www.rumah123.com//properti/balikpapan/hos14444729/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14444729/original/hos14444729-rumah-di...,"DI RUMAH 2 1/2 LANTAI STRATEGIS PINGGIR JALAN RAYA DR SUTOMO (KARANG REJO), BALIKPAPAN","Rp 2,2 Miliar","Karang Rejo, Balikpapan",7,7,176 m²,240 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Tidak,Tidak,Unfurnished,3,Perkotaan,Tidak,2 Mobil,NaN,Tidak,Bagus,Dijual
3,"DI RUMAH 2 1/2 LANTAI STRATEGIS PINGGIR JALAN RAYA DR SUTOMO (KARANG REJO), BALIKPAPAN",https://www.rumah123.com//properti/balikpapan/hos14444728/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14444728/original/hos14444728-rumah-di...,"DI RUMAH 2 1/2 LANTAI STRATEGIS PINGGIR JALAN RAYA DR SUTOMO (KARANG REJO), BALIKPAPAN","Rp 2,2 Miliar","Karang Rejo, Balikpapan",7,7,176 m²,240 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Tidak,Tidak,Unfurnished,3,Perkotaan,Tidak,2 Mobil,NaN,Tidak,Bagus,Dijual
4,BALIKPAPAN RUMAH DI PERUMAHAN KUTAI HILLS (HM173)TURUN HARGA 750JT,https://www.rumah123.com//properti/balikpapan/hos14209818/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14209818/original/hos14209818-rumah-di...,BALIKPAPAN RUMAH DI PERUMAHAN KUTAI HILLS (HM173)TURUN HARGA 750JT,Rp 850 Juta,"Balikpapan Utara, Balikpapan",3,2,120 m²,90 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,NaN,Perkotaan,Tidak,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
5,BALIKPAPAN RUMAH PERUMAHAN BUKIT DAMAI INDAH BDI (HM174),https://www.rumah123.com//properti/balikpapan/hos14209306/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14209306/original/hos14209306-rumah-di...,BALIKPAPAN RUMAH PERUMAHAN BUKIT DAMAI INDAH BDI (HM174),"Rp 1,9 Miliar","Balikpapan Selatan, Balikpapan",4,3,214 m²,120 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,Tidak,Tidak,Semi Furnished,NaN,Perkotaan,Tidak,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
6,73 Balikpapan Bima Jamrud Rumah Dalam Perumahan Padat Penduduk,https://www.rumah123.com//properti/balikpapan/hos14155522/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1737807/2023-12-05-12-22-58-70edf...,73 Balikpapan Bima Jamrud Rumah Dalam Perumahan Padat Penduduk,Rp 925 Juta,"Balikpapan Selatan, Balikpapan",2,1,92 m²,60 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
7,Rumah elit gaya minimalis modern sejuk diperumahan megah,https://www.rumah123.com//properti/balikpapan/hos14004571/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1655652/2023-06-14-10-05-42-d5f8a...,Rumah elit gaya minimalis modern sejuk diperumahan megah,"Rp 1,97 Miliar","Balikpapan Baru, Balikpapan",3,2,162 m²,110 m²,1,Rumah,SHM - Sertifikat Hak M

In [ ]:
# check null values
df_fix_combine.isnull().sum()

judul                     0
hyperlink                 0
img_hyperlinks            0
Judul                  1425
Harga                  1425
Lokasi                 1425
Kamar Tidur            1466
Kamar Mandi            1466
Luas Tanah             1466
Luas Bangunan          1466
Carport                1466
Tipe Properti          1466
Sertifikat             1466
Daya Listrik           1466
Kamar Pembantu         1466
Dapur                  1466
Ruang Makan            1466
Ruang Tamu             1466
Kondisi Perabotan      1466
Jumlah Lantai          1466
Pemandangan            1466
Terjangkau Internet    1466
Lebar Jalan            1466
Sumber Air             1466
Hook                   1466
Kondisi Properti       1466
Tipe Iklan             1466
dtype: int64

In [ ]:
# to csv
df_fix_combine.to_csv('./datasets/rumah123_fix_data_fixxx_part_2.csv')